In [1]:
import pandas as pd
import numpy as np
import io
import dvc.api
# from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os

import warnings; warnings.simplefilter('ignore')

In [ ]:
# !pip install dvc[gdrive]

In [ ]:
# !git clone https://github.com/isaaclucky/sales-prediction.git 

Cloning into 'sales-prediction'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (224/224), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 224 (delta 75), reused 153 (delta 34), pack-reused 0
Receiving objects: 100% (224/224), 2.97 MiB | 7.16 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [ ]:
# cd sales-prediction/

/content/sales-prediction


In [2]:
# !dvc pull 

In [61]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow 
import mlflow.sklearn

In [ ]:
# path="data/df_test.csv"
# repo='../'
# version="V1.0"

# data_url = dvc.api.read(path=path,
#                     repo=repo,
#                     rev=version
#                     )

In [37]:


import numpy as np 
import pandas as pd

In [57]:
np.random.randint(0,4)

0

In [3]:


df_train = pd.read_csv('../data/df_train_prep.csv')
df_test = pd.read_csv('../data/df_test_prep.csv')

/home/owon/anaconda3/envs/issue/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/owon/anaconda3/envs/issue/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [60]:
df_train.StateHoliday = df_train.StateHoliday.astype('string')
df_test.StateHoliday = df_test.StateHoliday.astype('string')

In [62]:
X_train = df_train.drop(columns=['Sales','Customers','Date','Unnamed: 0'],axis=1).copy(deep=True)
Y_train = df_train['Sales'].copy(deep=True)
X_test =df_test.drop(columns=['Unnamed: 0'],axis=1).copy(deep=True)

In [63]:
cont_cols = ['DayOfWeek', 'Until_Holiday', 'Since_Holiday','CompetitionOpenMonthDuration','CompetitionDistance','PromoOpenMonthDuration']
num_cols = list(X_train.select_dtypes(include = 'number').columns)
cat_cols = list(set(X_train.columns)-set(num_cols))
disc_cols = list(set(num_cols) - set(cont_cols))


In [78]:
if 'a' in df_test.columns:
    print('a')

In [64]:
cont_pipeline = make_pipeline(
    SimpleImputer(strategy = 'median'),
    PowerTransformer(method = 'yeo-johnson', standardize = False),
    StandardScaler()
)
disc_pipeline = make_pipeline(
    SimpleImputer(strategy = 'constant', fill_value = -1),
    StandardScaler()
)
cat_pipeline = make_pipeline(
    SimpleImputer(strategy = 'constant', fill_value = 'unknown'),
    OneHotEncoder()
)


preprocessor = ColumnTransformer(
    transformers = [
        ('continuous', cont_pipeline, cont_cols),
        ('discrete', disc_pipeline, disc_cols),
        ('categorical', cat_pipeline, cat_cols)
    ]
)




In [70]:
from pickle import load
prep = load(open('../models/data_transformer.pkl', 'rb'))

In [68]:
# from pickle import dump
# dump(preprocessor, open('../models/data_transformer.pkl', 'wb'))

In [10]:
preprocessor.fit(X_train)
preprocessor.fit(X_test)


ColumnTransformer(transformers=[('continuous',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('powertransformer',
                                                  PowerTransformer(standardize=False)),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['DayOfWeek', 'Until_Holiday', 'Since_Holiday',
                                  'CompetitionOpenMonthDuration',
                                  'CompetitionDistance',
                                  'PromoOpenMonthDuration']),
                                ('discrete',
                                 Pipel...
                                 ['DayOfYear', 'Week', 'Quarter', 'Open', 'Day',
                                  'SchoolHoliday', 'Promo', 'Promo2', 

In [11]:

def eval_metrics(actual, pred, verbose=True):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    if verbose:
        print("\tRMSE Score is: {:.5%}".format(rmse))
        print("\tR2 Square Score is: {:.5%}".format(r2))
        print("\tMAE Score is: {:.5%}".format(mae))
    return {f'RMSE Score': rmse, f'R2_Squared': r2, f'MAE Score': mae}


In [12]:
# mlflow.autolog(log_input_examples=True, disable_for_unsupported_versions=True, silent=True)

regressors = [
    LinearRegression(),
    DecisionTreeRegressor(),
    RandomForestRegressor(max_depth=10,max_features=20,random_state=42),
]


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



In [14]:
y_train.shape,y_test.shape,y_val.shape

((651013,), (203442,), (162754,))

In [15]:
X_train.shape,X_test.shape,X_val.shape

((651013, 25), (203442, 25), (162754, 25))

In [ ]:

# Fit the pipeline with the data

with mlflow.start_run() as run:
for r in regressors:
    pipe = Pipeline(steps = [
        ('preprocessor', preprocessor),
        ('regressor', r)
    ])
    pipe = pipe.fit(X_train, y_train)

    train_score = pipe.score(X_train, y_train)
    valid_score = pipe.score(X_val, y_val)
    print(f"Regression algorithm: {pipe.named_steps['regressor'].__class__.__name__}")
    print(("Train Values"))
    valid_metrics = eval_metrics(y_val, pipe.predict(X_val))
    print("Test Values")
    test_score = pipe.score(X_test, y_test)
    test_metrics = eval_metrics(y_test, pipe.predict(X_test))
    print("\n---------------------------")

    mlflow.log_metric(f"{pipe.named_steps['regressor'].__class__.__name__} Valid Score", valid_score)
    mlflow.log_metrics(valid_metrics)
    mlflow.log_metric(f"{pipe.named_steps['regressor'].__class__.__name__} Test Score", test_score)
    mlflow.log_metrics(test_metrics)

Regression algorithm: LinearRegression
Train Values
	RMSE Score is: 249205.68809%
	R2 Square Score is: 58.37729%
	MAE Score is: 175004.45377%
Test Values
	RMSE Score is: 248099.32291%
	R2 Square Score is: 58.37844%
	MAE Score is: 174462.13045%

---------------------------
Regression algorithm: DecisionTreeRegressor
Train Values
	RMSE Score is: 118736.02387%
	R2 Square Score is: 90.55114%
	MAE Score is: 66355.67421%
Test Values
	RMSE Score is: 119444.61430%
	R2 Square Score is: 90.35282%
	MAE Score is: 66505.61094%

---------------------------
Regression algorithm: RandomForestRegressor
Train Values
	RMSE Score is: 216454.98924%
	R2 Square Score is: 68.59856%
	MAE Score is: 145213.17871%
Test Values
	RMSE Score is: 215240.23794%
	R2 Square Score is: 68.67334%
	MAE Score is: 144551.53420%

---------------------------


## HyperParameter Tuning


In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [19]:
X_train_trans = preprocessor.fit_transform(X_train)
X_test_trans = preprocessor.fit_transform(X_test)
X_val_trans = preprocessor.fit_transform(X_val)

In [ ]:
X_train_trans[:50000].shape

(50000, 41)

In [ ]:
grid_search.fit(X_train_trans[:5000], y_train[:5000])

Fitting 3 folds for each of 288 candidates, totalling 864 fits


GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [80, 90, 100, 110],
                         'max_features': [2, 3], 'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [8, 10, 12],
                         'n_estimators': [100, 200, 300, 1000]},
             verbose=2)

In [41]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 100,
 'max_features': 3,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 1000}

In [44]:
best_grid = grid_search.best_estimator_


In [49]:
best_grid.predict(X_train_trans[:10])

array([6812.49937351, 6139.23327166, 7747.05383121, 8087.33614263,
        850.97630024, 6043.45926552, 4044.08547344, 8122.45602519,
       5769.16250341,  444.16681961])

In [48]:
y_train[:10]

523458    5439
794695    4792
118156    6280
167458    5826
605432       0
913621    6401
965553    2416
203087    9513
296559    6398
344698       0
Name: Sales, dtype: int64

In [16]:
from datetime import datetime
def generate_model_name() -> str:
    now = datetime.now()
    dt_string = now.strftime("%d-%m-%Y_%H:%M:%S_")
    return dt_string

In [17]:
from pickle import dump 
def save_model(model) -> None:
    try:
        file_name = generate_model_name() +'.pkl'
        with open(file_name, 'wb') as handle:
            dump(model, handle)
    except Exception as e:
        print('Error while producing model')

In [64]:
save_model(grid_search.best_estimator_)

## Random Forest with Best Parameters

In [23]:
rf = RandomForestRegressor(bootstrap=True,max_depth=100,random_state=42,max_features=3,min_samples_leaf=3,min_samples_split=8,n_estimators=1000)

In [24]:
mlflow.set_experiment('tuned_rf_regressor')

with mlflow.start_run() as run:
    rf.fit(X_train_trans, y_train)

    train_score = pipe.score(X_train_trans, y_train)
    valid_score = pipe.score(X_val_trans, y_val)
    print(f"Regression algorithm: {pipe.named_steps['regressor'].__class__.__name__}")
    print(("Train Values"))
    valid_metrics = eval_metrics(y_val, pipe.predict(X_val_trans))
    print("Test Values")
    test_score = pipe.score(X_test_trans, y_test)
    test_metrics = eval_metrics(y_test, pipe.predict(X_test_trans))
    print("\n---------------------------")

    mlflow.log_metric(f"{pipe.named_steps['regressor'].__class__.__name__} Valid Score", valid_score)
    mlflow.log_metrics(valid_metrics)
    mlflow.log_metric(f"{pipe.named_steps['regressor'].__class__.__name__} Test Score", test_score)
    mlflow.log_metrics(test_metrics)

In [ ]:
save_model(rf)
